When building complex models we often need to share large sets of variables and we might want to initialize all of them in one place. And this can be done using tf.variable_scope() and tf.get_variable().

One way to do variables sharing is to use classes to create a model, where the classes take care of managing the variables they need. For a lighter solution, not involing classes, tensorflow provides a VariableScope mechanism that allows easily share named variables while constructing a graph.

Variable Scope mechanism in tensorflow consists of two main functions like below:
* tf.get_variable(<name>, <shape>, <initializer>): creates or returns a variable with a given name;
* tf.variable_scope(<scope_name>): manages namespaces for names passed to tf.get_variable().


In [4]:
#We need to understand how tf.get_variable() works.
import tensorflow as tf
# v = tf.get_variable(name, shape, dtype, initializer)
# case1: the scope is set for creating new variables, as evidenced by tf.get_variable_scope().reuse == False
# In this case, v will be a newly created tf.Variable. The full name of the created variable will be set to the 
# current variable scope name + the provided name and a check will be performed to ensure that no variables with
# this full name exists yet. if a variable with this name already exists, it will raise a ValueError.
#with tf.variable_scope("foo"):
#    v = tf.get_variable("v", [1])
#assert v.name == "foo/v:0"

# case2: the scope is set for reusing variables, as evidenced by tf.get_variable_scope().reuse == True
# In this case, the call will search for an already existing variable with name equal to 'curren variable scope
# name + the provided name. If no such variable exists, a ValueError will be raised. If the variable is found, it
# will be returned.
with tf.variable_scope("foo3"):
    v = tf.get_variable("v", [1])
with tf.variable_scope("foo3", reuse = True):
    v1 = tf.get_variable("v", [1])
assert v is v1

   Basics of tf.variable_scope():
   The primary function of variable scope is to carry a name that will be used as prefix for variable names and
   a reuse-flag to distinguish the two cases described above. Nesting variable scope appends their names in a 
   way analogous to how directories work.

In [5]:
with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        v = tf.get_variable("v", [1])
        # The current variable scope can be retrieved using tf.get_variable_scope() and the reuse flag of the
        # current variable scope can be set to True by calling 'tf.get_variable_scope().reuse_variables()'
        #
        # Note: we can't set the reuse flag to False. The reason behind this is to allow to compose functions that
        # create models.
        tf.get_variable_scope().reuse_variables()
        v1 = tf.get_variable("v", [1])
assert v.name == "foo/bar/v:0"


In [7]:
with tf.variable_scope("root"):
    # At start, the scope is not reusing.
    assert tf.get_variable_scope().reuse == False
    with tf.variable_scope("foo"):
        # Opened a sub-scope, still not reusing.
        assert tf.get_variable_scope().reuse == False
    with tf.variable_scope("foo", reuse=True):
        # Explicitly opened a reusing scope.
        assert tf.get_variable_scope().reuse == True
        with tf.variable_scope("bar"):
            # Now sub-scope inherits the reuse flag.
            assert tf.get_variable_scope().reuse == True
    # Exited the reusing block, back to a non-reusing one.
    assert tf.get_variable_scope().reuse == False

In [9]:
# We can share parameters only by using variable with their names agreed, that is because we opened a reusing
# variable scope with exactly the same string. In more complex cases, it is useful to pass a VariableScope
# object rather than rely on getting the names right.
with tf.variable_scope("foo1") as foo_scope:
    v = tf.get_variable("v", [1])
with tf.variable_scope(foo_scope):
    w = tf.get_variable("w", [1])
with tf.variable_scope(foo_scope, reuse=True):
    v1 = tf.get_variable("v", [1])
    w1 = tf.get_variable("w", [1])
assert v1 is v and w1 is w

In [10]:
# When opening a variable scope using a previously existing scope we jump out of the current variable scope
# prefix to an entirely different one. This is fully independent of where we do it.
with tf.variable_scope("foo2") as foo2_scope:
    assert foo2_scope.name == "foo2"
with tf.variable_scope("bar"):
    with tf.variable_scope("baz") as other_scope:
        assert other_scope.name == "bar/baz"
        with tf.variable_scope(foo2_scope):
            assert foo2_scope.name == "foo2"

In [21]:
# variable scope can carry a default initializer. It's inherited by sub-scopes and passed to each tf.get_variable()
# call. But it will be overridden if another initializer is specified explicitly.
sess = tf.Session()
with tf.variable_scope("foo9", initializer = tf.constant_initializer(0.4)):
    #v = tf.get_variable("v", [1])
    #print v
    #print sess.run(v)
    #assert v.eval(session=sess) == 0.4 #Default initializer as set above
    w = tf.get_variable("w", [1], initializer = tf.constant_initializer(0.3))
    #assert w.eval(session=sess) == 0.3 #Specific initializer overrides the default
    with tf.variable_scope("bar"):
        v = tf.get_variable("v", [1])
        #assert v.eval(session=sess) == 0.4    #Inherited default initializer
    with tf.variable_scope("baz", initializer = tf.constant_initializer(0.2)):
        v = tf.get_variable("v", [1])
        #assert v.eval(session=sess) == 0.2 # Changed default initializer

In [27]:
# tf.variable_scope also influences the names of other ops in the scope. When we do with tf.variable_scope("name")
# , this implicitly opens a tf.name_scope("name"). like below:
with tf.variable_scope("boo1"):
    x = 1.0 + tf.get_variable("v", [1])
print x.op.name
assert x.op.name == "boo1/add"

# Name scopes can be opened in addition to a variable scope, and they will only affect the names of the ops, but
# not of variables.
with tf.variable_scope("boo1", reuse=True):
    with tf.name_scope("bar"):
        v = tf.get_variable("v", [1])
        x = 1.0 + v
assert v.name == "boo1/v:0"
assert x.op.name == "boo1/bar/add"

boo1/add


AssertionError: 

When opening a variable scope using a captured object instead of a string, we do not alter the current name scope for ops.
In particular, variable scope is heavily used for recurrent neural networks and sequence-to-sequence models.